<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/lecture-idea/10_root_finding/40_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
# This cell is for the Google Colaboratory
# https://stackoverflow.com/a/63519730
if 'google.colab' in str(get_ipython()):
  path_py = '/content/nmisp_py'

  import os
  if not os.path.exists(path_py):
    import subprocess
    subprocess.run(
        ('git', 'clone', 'https://github.com/kangwonlee/nmisp_py')
    )
  assert os.path.exists(path_py)

  import sys
  sys.path.insert(0, path_py)



# 적용 사례<br>Examples



In [ ]:
import pylab as py



## 경사 곡선로의 자동차<br>A car on a banked curve



ref : 
* Beer et. al., Vector Mechanics for Engineers, 8th Ed in SI, Dynamics, Sample problem 12.6, McGraw-Hill, 2007.
* http://www.gdecarli.it/php/circuit.php?var1=794&var2=2



[![NASCAR Racing](https://i.ytimg.com/vi/AYvdS73abgI/hqdefault.jpg)](https://youtu.be/AYvdS73abgI)


### 질문<br>Question



On a curved road with radius $\rho=315m$ and bank angle $\theta=31^\circ$, what is the speed $v$(m/s) such that no lateral friction forces would exert on the vehicle?<br>
반경 $\rho=315m$, 횡경사각 $\theta=31^\circ$ 인 곡선로에서 횡방향 마찰력 없이 차량이 달릴 수 있는 속도 $v$(m/s) 는?



### 풀이 예<br>One possible answer



지면에 수직한 중력 가속도 성분<br>
Component of the gravitational acceleration normal to the ground surface



$$
    g_n = g \cdot cos\theta
$$



지면에 평행한 중력 가속도 성분<br>
Component of the gravitational acceleration parallel to the ground surface



$$
    g_p = g \cdot sin\theta
$$



원심가속도<br>
Centrifugal Acceleration



$$
    a_c = \frac{v^2}{\rho}
$$



지면에 평행한 원심 가속도 성분<br>
Component of the centrifugal Acceleration normal to the ground surface



$$
    a_{cp} = a_c \cdot cos\theta
$$



지면에 수직인 원심 가속도 성분<br>
Component of the centrifugal acceleration parallel to the ground surface



$$
    a_{cn} = a_c \cdot sin\theta
$$



두 가속도의 지면에 평행한 성분들끼리 같으면 횡가속도는 0이 될 것이다.<br>
If two components of accelerations parallel to the ground are the same, the lateral acceleration would be zero.



$$
\begin{align}
g_p & = a_{cp} \\
gsin\theta & = \frac{v^2}{\rho}cos\theta \\
gsin\theta & - \frac{v^2}{\rho}cos\theta = 0
\end{align}
$$



매개변수<br>
Parameters



In [ ]:
g_mps2 = 9.8
theta_deg = 31
theta_rad = py.deg2rad(theta_deg)
rho_m = 315



$ f(v)=0 $



In [ ]:
def f(v_mps):
    return g_mps2 * py.sin(theta_rad) - v_mps * v_mps * py.cos(theta_rad) / rho_m



1변수 방정식 해법 함수를 불러들임



In [ ]:
import root_finding as rf



순차법<br>
Sequential Method



In [ ]:
%time v_seq_mps = rf.sequential(f, 0, 1/2**12, 1e-3, 20)
v_seq_mps



이분법<br>
Bisection Method



In [ ]:
%time v_bis_mps = rf.bisection(f, 0, 100, 1e-3)
v_bis_mps



뉴튼랩슨법<br>
Newotn Raphson Method



$$
\begin{align}
f(v) &= gsin\theta - \frac{v^2}{\rho}cos\theta \\
\frac{d}{dv}f(v) &= - 2\frac{v}{\rho}cos\theta
\end{align}
$$



In [ ]:
def df(v_mps):
    return - 2 * v_mps * py.cos(theta_rad) / rho_m



In [ ]:
%time v_nr_mps = rf.newton_raphson(f, df, 1, 1e-3)
v_nr_mps



## 여러 구간으로 나누어진 연속 함수<br>A piecewise continuous function



$$
h(x) = p(x) - \sqrt{2}
$$



$$
p(x) = 
\begin{cases}
x & x < -1 \\
sin \frac{\pi}{2} x & -1 \leq x < 1 \\
x & 1 < x
\end{cases}
$$



아래 함수는 float 형 매개변수를 받아들일 것이다<br>
    Ths function below expects a float argument<br>
반환형은 float형 일 것이다.<br>
    Also expected to return a float variable.



In [ ]:
def h(x: float) -> float:
    return p(x) - 2 ** (-0.5)



이 경우 `p(x)` 함수는 `h(x)` 함수를 호출 하기 전에 정의되면 된다.<br>
In this case, we need to define the `p(x)` function before calling `h(x).



In [ ]:
def p(x: float) -> float:
    if (-1 > x) or (1 < x):
        result = x
    else :
        result = py.sin(py.pi * x * 0.5)

    return result



In [ ]:
x = py.linspace(-2, 2)
y = py.array([h(xi) for xi in x])

py.plot(x, y, '.')
py.grid(True)
py.xlabel('x')
py.ylabel('h(x)')



순차법<br>
Sequential Method



In [ ]:
%time x_seq = rf.sequential(h, 0, 1/2**12, 1e-3, 1)
x_seq



이분법<br>
Bisection Method



In [ ]:
%time x_bis = rf.bisection(h, 0, 3, 1e-3)
x_bis



뉴튼법<br>
Newton-Raphson Method



함수 $h(x)$ 는 $x=\pm 1$ 에서 기울기가 불연속이다.<br>
The slope of the function $h(x)$ is not continuous at $x=\pm 1$.



기울기를 구하기 위해 아래 함수를 사용해 보자.<br>
To find the slopes, let's use following function.



$$
q(x) = 
\begin{cases}
1 & x < -1 \\
\frac{\pi}{2} cos \frac{\pi}{2} x & -1 \leq x < 1 \\
1 & 1 < x
\end{cases}
$$



In [ ]:
def q(x: float) -> float:
    if (-1 > x) or (1 < x):
        result = 1
    else :
        result = (py.pi * 0.5) * py.sin(py.pi * x * 0.5)

    return result



In [ ]:
%time x_nr = rf.newton_raphson(h, q, .1, 1e-3)
x_nr



## 유한한 구간에만 정의된 함수<br>A function defined within a finite interval



$$
r(x) = 2\sqrt{1-\frac{x^2}{4^2}} - \frac{1}{2}
$$



함수 $r(x)$ 는 아래 구간에서만 정의된다.<br>
Function $r(x)$ is defined within the following interval only.



$$ -4 \leq x \leq 4 $$



In [ ]:
def r(x: float) -> float:
    return 2 * (1 - (x**2)/16) ** 0.5 - 0.5



In [ ]:
x = 4 * py.cos(py.deg2rad(py.arange(180, 0-1, -1)))

py.plot(x, r(x), '.')
py.grid(True)
py.xlabel('x')
py.ylabel('r(x)')
py.axis('equal');



순차법<br>
Sequential Method



In [ ]:
%time x_seq_interval = rf.sequential(r, 0, 1/2**12, 1e-3, 3.9)
x_seq_interval



이분법<br>
Bisection Method



In [ ]:
%time x_bis_interval = rf.bisection(r, 0, 3.9, 1e-3)
x_bis_interval



뉴튼법을 위한 $r(x)$ 함수의 미분<br>
Derivative of $r(x)$ for the Newton Raphson method



In [ ]:
import sympy as sym

x = sym.symbols('x', real=True)
r_sym = 2 * sym.sqrt(1 - (x/4)**2) -  1 / 2



In [ ]:
r_sym.diff(x)



In [ ]:
print(sym.python(r_sym.diff(x)))



In [ ]:
def drdx(x:float) -> float:
    return float((-x / 8) * (1 - (x**2)/16) ** (-0.5))



그러나, 함수 `drdx()` 는 $\left| x \right| > 4$ 일 경우 예외 Exception 을 발생시킬 것이다.<br>However, function `drdx()` would raise an Exception if  $\left| x \right| > 4$.



In [ ]:
# try:
#     %time x_nr_interval = rf.newton_raphson(r, drdx, .1, 1e-3)
#     x_nr_interval
# except TypeError as te:
#     print(te)
# except NameError as ne:
#     print(ne)



## 여러 구간으로 나누어진 불연속 함수<br>A piecewise discontinuous function



$$
s(x) = t(x) - \sqrt{2}
$$



$$
t(x) = 
\begin{cases}
x & x < -1 \\
sin \pi x & -1 \leq x < 1 \\
x & 1 < x
\end{cases}
$$



In [ ]:
def s(x: float) -> float:
    return t(x) - 2 ** (-0.5)



In [ ]:
def t(x: float) -> float:
    if (-1 > x) or (1 < x):
        result = x
    else :
        result = py.sin(py.pi * x)

    return result



In [ ]:
x = py.linspace(-2, 2)
y = py.array([s(xi) for xi in x])

py.plot(x, y, '.')
py.grid(True)
py.xlabel('x')
py.ylabel('s(x)')



순차법<br>
Sequential Method



In [ ]:
%time x_seq = rf.sequential(s, 0, 1/2**12, 1e-3, 1)
x_seq



이분법<br>
Bisection Method



In [ ]:
%time x_bis = rf.bisection(s, 0, 0.51, 1e-3)
x_bis



뉴튼법<br>
Newton-Raphson Method



함수 $h(x)$ 는 $x=\pm 1$ 에서 기울기가 불연속이다.<br>
The slope of the function $h(x)$ is not continuous at $x=\pm 1$.



기울기를 구하기 위해 아래 함수를 사용해 보자.<br>
To find the slopes, let's use following function.



$$
q(x) = 
\begin{cases}
1 & x < -1 \\
\pi cos \pi x & -1 \leq x < 1 \\
1 & 1 < x
\end{cases}
$$



In [ ]:
def u(x: float) -> float:
    if (-1 > x) or (1 < x):
        result = 1
    else :
        result = py.pi * py.sin(py.pi * x)

    return result



In [ ]:
%time x_nr = rf.newton_raphson(s, u, .1, 1e-3)
x_nr



In [ ]:
%time x_nr = rf.newton_raphson(s, u, .9, 1e-3)
x_nr



## 연습 문제<br>Exercises



도전과제 1 : 반지름이 다음과 같을 때, 시속 160 km의 차량이 횡방향 마찰력 없이 주행할 수 있는 경사각($^\circ$)은?<br>
Try this 1 : With following radius, what would be the bank angle ($^\circ$) such that a vehicle of 160km/hr can travel without lateral friction force?



$$
\rho = 200(m)
$$



## Final Bell<br>마지막 종



In [ ]:
# stackoverfow.com/a/24634221
import os
os.system("printf '\a'");

